# 基于私有数据的QA系统

步骤：
1. 处理数据，分成合适的大小片段
2. 为每个片段创建Embedding
3. 把embedding存储到向量数据库，供后续查询使用
4. 获取用户输入
5. 从数据库中查询与用户输入相似度比较高的片段
6. 把以上片段作为参考数据注入到Prompt里，提交给LLM
7. 把结果返回给用户

## 处理数据，建立向量数据库
- https://github.com/pgvector/pgvector - Postgres的插件
- FAISS
- Pinecorn
- Chroma

### 处理数据

In [2]:
# 加载md文件
from pathlib import Path

ps = list(Path("dataset/dev-guide").glob("**/*.md"))
data = []
sources = []
for p in ps:
    with open(p) as f:
        data.append(f.read())
    sources.append(p)
print(sources)

[PosixPath('dataset/dev-guide/payment.md'), PosixPath('dataset/dev-guide/workflow.md'), PosixPath('dataset/dev-guide/version.md'), PosixPath('dataset/dev-guide/swagger.md'), PosixPath('dataset/dev-guide/branch.md'), PosixPath('dataset/dev-guide/commit.md'), PosixPath('dataset/dev-guide/_index.md')]


In [4]:
from langchain.text_splitter import CharacterTextSplitter

# 把文档拆成小块，方便LLMs使用
text_splitter = CharacterTextSplitter(chunk_size=1500, separator="\n")
docs = []
metadatas = []
for i, d in enumerate(data):
    splits = text_splitter.split_text(d)
    docs.extend(splits)
    metadatas.extend([{"source": sources[i]}] * len(splits))

import tiktoken
token_encoder = tiktoken.encoding_for_model('gpt-3.5-turbo')
sizes = [len(token_encoder.encode(t)) for t in docs]
# $0.0016 / 1K tokens
PRIZE = 0.0016/1024
print(f"Docs: {len(docs)}, 总Token: {sum(sizes)}, 花费 ${sum(sizes)*PRIZE}")

Docs: 8, 总Token: 3906, 花费 $0.006103125


### 建立数据库

In [8]:
import faiss
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import pickle

# 创建一个本地的向量数据库（文件）
store = FAISS.from_texts(docs, OpenAIEmbeddings(), metadatas=metadatas)
faiss.write_index(store.index, "dataset/docs.index")
store.index = None
with open("dataset/faiss_store.pkl", "wb") as f:
    pickle.dump(store, f)

### 使用数据库查询

In [15]:
import faiss
# from langchain import OpenAI
# from langchain.chains import VectorDBQAWithSourcesChain
from langchain.chains import RetrievalQAWithSourcesChain
import pickle
from langchain.chat_models import PromptLayerChatOpenAI

chat = PromptLayerChatOpenAI(pl_tags=["note-qa-withsourcechain"], temperature=0, verbose=True)

index = faiss.read_index("dataset/docs.index")

with open("dataset/faiss_store.pkl", "rb") as f:
    store = pickle.load(f)

store.index = index
# chain = VectorDBQAWithSourcesChain.from_llm(llm=chat, vectorstore=store)
chain = RetrievalQAWithSourcesChain.from_llm(llm=chat, retriever=store.as_retriever())

#### 问题一

In [17]:
question = "开发流程有哪些阶段？"
result = chain({"question": question})
print(f"Answer: {result['answer']}")
print(f"Sources: {result['sources']}")

Answer: 开发流程有五个阶段，分别是需求阶段、设计阶段、开发阶段、验收阶段和发布阶段。

Sources: dataset/dev-guide/workflow.md


#### 问题二

In [16]:
question = "开发流程有哪些阶段？每个阶段的产出是什么？请使用中文回答。"
result = chain({"question": question})
print(f"Answer: {result['answer']}")
print(f"Sources: {result['sources']}")

Answer: 开发流程有五个阶段，分别是需求阶段、设计阶段、开发阶段、验收阶段和发布阶段。每个阶段的产出如下：
- 需求阶段：产品文档、交互设计和测试用例。
- 设计阶段：多语言表、UI设计稿、测试用例、API文档、技术文档和研发排期表。
- 开发阶段：程序、自测报告和部署文档。
- 验收阶段：测试报告、版本(CI自动生成)和验收环境。
- 发布阶段：版本报告、正式环境和更新申请。
- 运营阶段：运维报告。

Sources: dataset/dev-guide/workflow.md, dataset/dev-guide/payment.md, dataset/dev-guide/_index.md, dataset/dev-guide/branch.md


#### 问题三

In [19]:
question = "开发流程有哪些阶段？每个阶段的产出是什么？"
result = chain({"question": question})
print(f"Answer: {result['answer']}")
print(f"Sources: {result['sources']}")

Answer: 开发流程有五个阶段：需求阶段、设计阶段、开发阶段、验收阶段和发布阶段。每个阶段的产出如下：
- 需求阶段：产品文档、交互设计、测试用例
- 设计阶段：多语言表、UI设计稿、测试用例、API文档、技术文档、研发排期表
- 开发阶段：程序、自测报告、部署文档
- 验收阶段：测试报告、版本(CI自动生成)、验收环境
- 发布阶段：版本报告、正式环境、更新申请
- 运营阶段：运维报告

Sources: dataset/dev-guide/workflow.md


#### 问题四

In [18]:
question = "开发流程每个阶段的产出是什么？"
result = chain({"question": question})
print(f"Answer: {result['answer']}")
print(f"Sources: {result['sources']}")

Answer: - 需求阶段的产出是产品文档、交互设计、测试用例。
- 设计阶段的产出是多语言表、UI设计稿、测试用例、API文档、技术文档、研发排期表。
- 开发阶段的产出是程序、自测报告、部署文档、生成测试版本。
- 验收阶段的产出是测试报告、版本(CI自动生成)、验收环境。
- 发布阶段的产出是版本报告、正式环境、更新申请。
- 运营阶段的产出是运维报告。

Sources: dataset/dev-guide/workflow.md, dataset/dev-guide/branch.md


#### 总结
在PromptLayer中查看记录，可以了解langchain的工作逻辑：
1. 在向量数据库中查询相关文档
2. 对于每个文档，让LLM总结出和问题相关的部分
3. 汇总2处理的结果，输出最终结果

根据例子二和例子三，也会发现问题越发散，解决起来越麻烦，只是加了句和问题无关的内容，多查了三个文档。

如何精准、简洁的提问，会变得越来越重要。

## 参考资料

- ClippyGPT - How I Built Supabase’s OpenAI Doc Search (Embeddings) - https://www.youtube.com/watch?v=Yhtjd7yGGGA
- https://github.com/hwchase17/notion-qa